# Data download via API from Alpha Vantage

In [20]:
# Fetch raw data inlucding income statement, balance sheet and cash flow from AlphaVantage API
# This will allow us to calculate a set of metrics which can help in gauging the health of a bank or firm

import requests
import time
import pandas as pd
from datetime import datetime

# --- Configuration ---
API_KEY = 'FU4V5XSIS7W5AUDE'
BASE_URL = 'https://www.alphavantage.co/query?'
CALL_DELAY = 15 # seconds, to stay within 5 calls per minute (3 API calls per bank * 15s = 45s per bank)

# Dictionary of banks and their ticker symbols
BANKS = {
    'JPMorgan Chase': 'JPM',
    'HSBC Holdings': 'HSBC',
    'Credit Suisse': 'CS', # Note: Acquired by UBS in June 2023. Data for later periods may be incomplete or under UBS.
    'UniCredit': 'UCG',
    'ING Groep': 'ING',
    'Santander': 'SAN',
    'Barclays': 'BCS',
    'Deutsche Bank': 'DB'
}

START_DATE = datetime(2022, 1, 1)
END_DATE = datetime(2025, 12, 31)

# --- Helper function to fetch data ---
def fetch_alpha_vantage_data(function, symbol, api_key):
    url = f"{BASE_URL}function={function}&symbol={symbol}&apikey={api_key}"
    try:
        response = requests.get(url)
        response.raise_for_status() # Raise an exception for bad status codes
        data = response.json()
        time.sleep(CALL_DELAY) # Wait to respect rate limit
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {function} data for {symbol}: {e}")
        return None

# --- Main script execution for all banks ---
for bank_name, symbol in BANKS.items():
    print(f"\n--- Processing {bank_name} ({symbol}) ---")
    all_quarterly_data = []

    # Fetch Income Statement
    print(f"  Fetching Income Statement for {symbol}...")
    income_statement_data = fetch_alpha_vantage_data("INCOME_STATEMENT", symbol, API_KEY)
    if income_statement_data and 'quarterlyReports' in income_statement_data:
        for report in income_statement_data['quarterlyReports']:
            fiscal_date_str = report.get('fiscalDateEnding')
            if fiscal_date_str:
                fiscal_date = datetime.strptime(fiscal_date_str, '%Y-%m-%d')
                if START_DATE <= fiscal_date <= END_DATE:
                    report_data = {k: v for k, v in report.items() if v not in ['', 'None']}
                    report_data['source'] = 'Income Statement'
                    all_quarterly_data.append(report_data)

    # Fetch Balance Sheet
    print(f"  Fetching Balance Sheet for {symbol}...")
    balance_sheet_data = fetch_alpha_vantage_data("BALANCE_SHEET", symbol, API_KEY)
    if balance_sheet_data and 'quarterlyReports' in balance_sheet_data:
        for report in balance_sheet_data['quarterlyReports']:
            fiscal_date_str = report.get('fiscalDateEnding')
            if fiscal_date_str:
                fiscal_date = datetime.strptime(fiscal_date_str, '%Y-%m-%d')
                if START_DATE <= fiscal_date <= END_DATE:
                    report_data = {k: v for k, v in report.items() if v not in ['', 'None']}
                    report_data['source'] = 'Balance Sheet'
                    all_quarterly_data.append(report_data)

    # Fetch Cash Flow (less directly relevant for requested metrics, but good for completeness)
    print(f"  Fetching Cash Flow for {symbol}...")
    cash_flow_data = fetch_alpha_vantage_data("CASH_FLOW", symbol, API_KEY)
    if cash_flow_data and 'quarterlyReports' in cash_flow_data:
        for report in cash_flow_data['quarterlyReports']:
            fiscal_date_str = report.get('fiscalDateEnding')
            if fiscal_date_str:
                fiscal_date = datetime.strptime(fiscal_date_str, '%Y-%m-%d')
                if START_DATE <= fiscal_date <= END_DATE:
                    report_data = {k: v for k, v in report.items() if v not in ['', 'None']}
                    report_data['source'] = 'Cash Flow'
                    all_quarterly_data.append(report_data)

    if not all_quarterly_data:
        print(f"  No data retrieved for {bank_name} ({symbol}) for the specified period. Skipping.")
        continue

    # Create a DataFrame from the collected raw data
    raw_df = pd.DataFrame(all_quarterly_data)

    # Convert relevant columns to numeric, coercing errors to NaN
    numeric_cols = [
        'netIncome', 'totalAssets', 'netInterestIncome', 'interestIncome',
        'interestExpense', 'provisionForLoanAndLeaseLosses', 'totalShareholderEquity',
        'totalLiabilities', 'deposits', 'shortTermDebt', 'longTermDebt'
    ]
    for col in numeric_cols:
        if col in raw_df.columns:
            raw_df[col] = pd.to_numeric(raw_df[col], errors='coerce')

    # Group by fiscalDateEnding to consolidate metrics from different statements
    processed_reports = {}
    for _, row in raw_df.iterrows():
        date = row['fiscalDateEnding']
        if date not in processed_reports:
            processed_reports[date] = {'Fiscal Date Ending': date}

        # Merge data, handling potential duplicates by preferring certain sources
        for col, value in row.items():
            if col not in ['source', 'fiscalDateEnding']:
                # Prioritize Balance Sheet (2), then Income Statement (1), then Cash Flow (0)
                current_preference = processed_reports[date].get('source_preference', -1)
                new_preference = 0
                if row['source'] == 'Balance Sheet': new_preference = 2
                elif row['source'] == 'Income Statement': new_preference = 1

                if col not in processed_reports[date] or new_preference > current_preference:
                    processed_reports[date][col] = value
                    processed_reports[date]['source_preference'] = new_preference
                elif col in processed_reports[date] and new_preference == current_preference:
                    # If same preference and value is NaN, update if new value is not NaN
                    if pd.isna(processed_reports[date][col]) and pd.notna(value):
                        processed_reports[date][col] = value

    final_df_list_raw = []
    for date in sorted(processed_reports.keys()):
        report_data = processed_reports[date]
        # Keep only relevant columns for raw output, removing internal preference
        clean_report_data = {k: v for k, v in report_data.items() if k not in ['source_preference']}
        final_df_list_raw.append(clean_report_data)

    raw_consolidated_df = pd.DataFrame(final_df_list_raw)
    raw_consolidated_df = raw_consolidated_df.sort_values(by='Fiscal Date Ending').reset_index(drop=True)

    # Save raw consolidated data to Excel
    raw_output_file = f"{symbol}_Quarterly_Reports_2022_2025.xlsx"
    try:
        raw_consolidated_df.to_excel(raw_output_file, index=False)
        print(f"  Raw consolidated data saved to '{raw_output_file}'")
    except Exception as e:
        print(f"  Error saving raw data for {symbol} to Excel: {e}")


print("\n--- All banks processed. ---")
print("Please check the generated Excel files in your script's directory.")
print("\n--- Important Note on Data Limitations ---")
print("  - Specific bank regulatory metrics (e.g., CET1, LCR, NPL, explicit Wholesale Funding) are NOT available directly from Alpha Vantage's general fundamental data and are therefore not included in this analysis.")
print("  - The 'Credit Suisse' data might be incomplete or affected by its acquisition by UBS from mid-2023 onwards.")



--- Processing JPMorgan Chase (JPM) ---
  Fetching Income Statement for JPM...
  Fetching Balance Sheet for JPM...
  Fetching Cash Flow for JPM...
  Raw consolidated data saved to 'JPM_Quarterly_Reports_2022_2025.xlsx'

--- Processing HSBC Holdings (HSBC) ---
  Fetching Income Statement for HSBC...
  Fetching Balance Sheet for HSBC...
  Fetching Cash Flow for HSBC...
  Raw consolidated data saved to 'HSBC_Quarterly_Reports_2022_2025.xlsx'

--- Processing Credit Suisse (CS) ---
  Fetching Income Statement for CS...
  Fetching Balance Sheet for CS...
  Fetching Cash Flow for CS...
  No data retrieved for Credit Suisse (CS) for the specified period. Skipping.

--- Processing UniCredit (UCG) ---
  Fetching Income Statement for UCG...
  Fetching Balance Sheet for UCG...
  Fetching Cash Flow for UCG...
  No data retrieved for UniCredit (UCG) for the specified period. Skipping.

--- Processing ING Groep (ING) ---
  Fetching Income Statement for ING...
  Fetching Balance Sheet for ING...
  Fe

# Calculating various metrics from raw data

In [21]:
# Code to calculate derived financial metrics from quarterly report data derived from API data download
# Output is a set of separate files with the various metrics displayed for our Streamlit input

import requests
import time
import pandas as pd
from datetime import datetime

# Dictionary of banks and their ticker symbols (used for file naming)
BANKS = {
    'JPMorgan Chase': 'JPM',
    'HSBC Holdings': 'HSBC',
    'Credit Suisse': 'CS',
    'UniCredit': 'UCG',
    'ING Groep': 'ING',
    'Santander': 'SAN',
    'Barclays': 'BCS',
    'Deutsche Bank': 'DB'
}

# --- Function to perform analysis for a single bank ---
def analyze_bank_metrics(symbol, bank_name):
    input_file = f"{symbol}_Quarterly_Reports_2022_2025.xlsx"
    output_analysis_file = f"{symbol}_Extended_Fiscal_Health_Analysis.xlsx"

    print(f"\n--- Analyzing Extended Metrics for {bank_name} ({symbol}) ---")

    if not os.path.exists(input_file):
        print(f"  Error: Input file '{input_file}' not found. Please ensure the data fetching script ran successfully for this bank.")
        return

    try:
        df = pd.read_excel(input_file)
        print(f"  Successfully loaded data from '{input_file}'")
    except Exception as e:
        print(f"  Error loading data from '{input_file}': {e}")
        return

    # Insert the 'Bank Name' column at the beginning
    df.insert(0, 'Bank Name', bank_name)

    # --- Data Preparation ---
    df['Fiscal Date Ending'] = pd.to_datetime(df['Fiscal Date Ending'])
    df = df.sort_values(by='Fiscal Date Ending').reset_index(drop=True)

    df['Year'] = df['Fiscal Date Ending'].dt.year
    df['Quarter'] = df['Fiscal Date Ending'].dt.quarter

    # Convert all necessary columns to numeric, coercing errors
    numeric_cols = [
        'netIncome', 'totalAssets', 'netInterestIncome', 'interestIncome', 'interestExpense',
        'totalShareholderEquity', 'totalRevenue', 'operatingIncome', 'operatingExpenses',
        'totalCurrentAssets', 'totalCurrentLiabilities', 'cashAndCashEquivalentsAtCarryingValue',
        'totalLiabilities', 'commonStockSharesOutstanding', 'goodwill', 'intangibleAssets',
        'shortTermDebt', 'longTermDebt', 'shortLongTermDebtTotal'
    ]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Handle potential missing Total Assets for NIM calculation if totalAssets column is missing,
    # though it should be present from previous data fetching.
    if 'totalAssets' not in df.columns or df['totalAssets'].isnull().all():
        print(f"  Warning: 'totalAssets' column is missing or entirely empty for {symbol}. NIM calculation might be affected.")
        df['totalAssets'] = 1 # Prevent division by zero for now, but NIM will be NaN

    # --- Calculate Derived Metrics ---
    # Profitability
    df['Calculated NIM'] = df['netInterestIncome']
    df['Calculated NIM'] = df.apply(
        lambda row: ((row['interestIncome'] - row['interestExpense']) / row['totalAssets']) * 100
        if pd.isna(row['Calculated NIM']) and pd.notna(row['interestIncome']) and pd.notna(row['interestExpense']) and row['totalAssets'] != 0
        else row['Calculated NIM'], axis=1
    )
    df['Calculated ROE'] = (df['netIncome'] / df['totalShareholderEquity']) * 100
    df['Calculated Operating Margin'] = (df['operatingIncome'] / df['totalRevenue']) * 100
    df['Calculated Net Profit Margin'] = (df['netIncome'] / df['totalRevenue']) * 100

    # Efficiency
    df['Calculated Cost-to-Income Ratio'] = (df['operatingExpenses'] / df['totalRevenue']) * 100

    # Leverage/Solvency (Accounting-based)
    df['Calculated Debt-to-Equity Ratio'] = df['totalLiabilities'] / df['totalShareholderEquity']
    df['Calculated Equity-to-Assets Ratio'] = df['totalShareholderEquity'] / df['totalAssets']

    # Liquidity (General)
    df['Calculated Current Ratio'] = df['totalCurrentAssets'] / df['totalCurrentLiabilities']
    df['Calculated Cash Ratio'] = df['cashAndCashEquivalentsAtCarryingValue'] / df['totalCurrentLiabilities']


    # Define metrics for analysis, including newly calculated ones
    metrics_to_analyze = {
        'netIncome': 'Net Income',
        'Calculated NIM': 'Net Interest Margin (NIM)',
        'Calculated ROE': 'Return on Equity (ROE)',
        'Calculated Operating Margin': 'Operating Margin',
        'Calculated Net Profit Margin': 'Net Profit Margin',
        'Calculated Cost-to-Income Ratio': 'Cost-to-Income Ratio',
        'Calculated Debt-to-Equity Ratio': 'Debt-to-Equity Ratio',
        'Calculated Equity-to-Assets Ratio': 'Equity-to-Assets Ratio',
        'Calculated Current Ratio': 'Current Ratio',
        'Calculated Cash Ratio': 'Cash Ratio',
    }

    analysis_results = []

    # --- Perform Analysis for each Metric ---
    for metric_col, metric_name_display in metrics_to_analyze.items():
        if metric_col not in df.columns:
            print(f"  Warning: Metric column '{metric_col}' not found in DataFrame for {symbol}. Skipping analysis for it.")
            continue

        # Calculate Quarter-over-Quarter % Change
        df[f'{metric_col}_QoQ_Change'] = df[metric_col].pct_change() * 100

        for i, row in df.iterrows():
            # Skip the first quarter as there's no previous quarter for % change
            if i == 0:
                continue

            fiscal_year = row['Year']
            fiscal_quarter = row['Quarter']
            actual_value = row[metric_col]
            qoq_change = row[f'{metric_col}_QoQ_Change']
            assessment = "Neutral" # Default to Neutral

            # --- Health Assessment Logic ---
            positive_threshold = 5.0 # e.g., >5% growth is good
            negative_threshold = -5.0 # e.g., <-5% decline is bad

            if pd.notna(actual_value):
                if pd.isna(qoq_change):
                    assessment = "Neutral" # Can't determine trend without previous quarter
                else:
                    # Profitability metrics (higher is generally better)
                    if metric_col in ['netIncome', 'Calculated NIM', 'Calculated ROE', 'Calculated Operating Margin', 'Calculated Net Profit Margin']:
                        if actual_value > 0 and qoq_change > positive_threshold:
                            assessment = "Good"
                        elif actual_value > 0 and qoq_change < negative_threshold:
                            assessment = "Bad"
                        elif actual_value <= 0: # Negative income/profitability is generally bad
                            assessment = "Bad"
                        else:
                            assessment = "Neutral"
                    # Efficiency (lower is generally better for Cost-to-Income)
                    elif metric_col == 'Calculated Cost-to-Income Ratio':
                        if qoq_change < negative_threshold: # Significant decrease in ratio
                            assessment = "Good"
                        elif qoq_change > positive_threshold: # Significant increase in ratio
                            assessment = "Bad"
                        else:
                            assessment = "Neutral"
                    # Leverage (lower is generally better for Debt-to-Equity)
                    elif metric_col == 'Calculated Debt-to-Equity Ratio':
                        if qoq_change < negative_threshold: # Significant decrease in debt relative to equity
                            assessment = "Good"
                        elif qoq_change > positive_threshold: # Significant increase in debt relative to equity
                            assessment = "Bad"
                        else:
                            assessment = "Neutral"
                    # Solvency/Liquidity (higher is generally better for Equity-to-Assets, Current, Cash Ratios)
                    elif metric_col in ['Calculated Equity-to-Assets Ratio', 'Calculated Current Ratio', 'Calculated Cash Ratio']:
                        if qoq_change > positive_threshold:
                            assessment = "Good"
                        elif qoq_change < negative_threshold:
                            assessment = "Bad"
                        else:
                            assessment = "Neutral"
            else:
                assessment = "N/A (Data Missing)"

            analysis_results.append({
                'Bank Name': bank_name, # Add Bank Name here
                'Year': fiscal_year,
                'Quarter': fiscal_quarter,
                'Financial Metric Name': metric_name_display,
                'Actual Value': actual_value,
                '% Change (QoQ)': qoq_change,
                'Assessment': assessment
            })

    analysis_df = pd.DataFrame(analysis_results)
    analysis_df = analysis_df.sort_values(by=['Bank Name', 'Year', 'Quarter', 'Financial Metric Name']).reset_index(drop=True) # Sort by Bank Name

    # --- Output to Excel ---
    try:
        analysis_df.to_excel(output_analysis_file, index=False)
        print(f"  Extended fiscal health analysis saved to '{output_analysis_file}'")
    except Exception as e:
        print(f"  Error saving analysis for {symbol} to Excel: {e}")

# --- Run analysis for all banks ---
print("Starting extended fiscal health analysis for all banks...")
for bank_name, symbol in BANKS.items():
    analyze_bank_metrics(symbol, bank_name)

print("\n--- All extended analyses complete. ---")
print("Please check the generated Excel files in your script's directory, named like '[SYMBOL]_Extended_Fiscal_Health_Analysis.xlsx'.")
print("\n--- General Interpretation Guide ---")
print("  - 'Good': Generally indicates positive trends or strong performance for the metric.")
print("  - 'Neutral': Indicates stable, minor, or inconclusive changes.")
print("  - 'Bad': Generally indicates negative trends or weaker performance for the metric.")
print("  - '% Change (QoQ)': Quarter-over-Quarter percentage change.")
print("  - Thresholds for 'Good'/'Bad' are currently set at >5% growth or <-5% decline respectively and can be adjusted within the code.")
print("\n--- Important Note on Data Limitations ---")
print("  - This analysis uses publicly available fundamental data which does NOT directly represent highly specific, legally obliged regulatory metrics (like precise CET1, LCR, NSFR, NPL ratios as defined by Basel III).")
print("  - The 'Credit Suisse' data might be incomplete or affected by its acquisition by UBS from mid-2023 onwards.")


Starting extended fiscal health analysis for all banks...

--- Analyzing Extended Metrics for JPMorgan Chase (JPM) ---
  Successfully loaded data from 'JPM_Quarterly_Reports_2022_2025.xlsx'
  Extended fiscal health analysis saved to 'JPM_Extended_Fiscal_Health_Analysis.xlsx'

--- Analyzing Extended Metrics for HSBC Holdings (HSBC) ---
  Successfully loaded data from 'HSBC_Quarterly_Reports_2022_2025.xlsx'
  Extended fiscal health analysis saved to 'HSBC_Extended_Fiscal_Health_Analysis.xlsx'

--- Analyzing Extended Metrics for Credit Suisse (CS) ---
  Error: Input file 'CS_Quarterly_Reports_2022_2025.xlsx' not found. Please ensure the data fetching script ran successfully for this bank.

--- Analyzing Extended Metrics for UniCredit (UCG) ---
  Error: Input file 'UCG_Quarterly_Reports_2022_2025.xlsx' not found. Please ensure the data fetching script ran successfully for this bank.

--- Analyzing Extended Metrics for ING Groep (ING) ---
  Successfully loaded data from 'ING_Quarterly_Repor

/tmp/ipython-input-21-2166518395.py:114: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df[f'{metric_col}_QoQ_Change'] = df[metric_col].pct_change() * 100
/tmp/ipython-input-21-2166518395.py:114: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df[f'{metric_col}_QoQ_Change'] = df[metric_col].pct_change() * 100
/tmp/ipython-input-21-2166518395.py:114: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df[f'{me

  Extended fiscal health analysis saved to 'SAN_Extended_Fiscal_Health_Analysis.xlsx'

--- Analyzing Extended Metrics for Barclays (BCS) ---
  Successfully loaded data from 'BCS_Quarterly_Reports_2022_2025.xlsx'
  Extended fiscal health analysis saved to 'BCS_Extended_Fiscal_Health_Analysis.xlsx'

--- Analyzing Extended Metrics for Deutsche Bank (DB) ---
  Successfully loaded data from 'DB_Quarterly_Reports_2022_2025.xlsx'
  Extended fiscal health analysis saved to 'DB_Extended_Fiscal_Health_Analysis.xlsx'

--- All extended analyses complete. ---
Please check the generated Excel files in your script's directory, named like '[SYMBOL]_Extended_Fiscal_Health_Analysis.xlsx'.

--- General Interpretation Guide ---
  - 'Good': Generally indicates positive trends or strong performance for the metric.
  - 'Neutral': Indicates stable, minor, or inconclusive changes.
  - 'Bad': Generally indicates negative trends or weaker performance for the metric.
  - '% Change (QoQ)': Quarter-over-Quarter per